# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context of the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import plotly as py
import plotly.graph_objs as go

py.offline.init_notebook_mode(connected=True)

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [3]:
# total number of callbacks
n_t_calls = int(sum(data.call))
sample_p_t = n_t_calls / len(data.call) * 100 # working with percentages
print("The total number of callbacks is " + str(n_t_calls) + " out of " + str(len(data.call)) 
      + " potential candidates (p-hat = " + str(round(sample_p_t, 2)) +"%).")

# number of callbacks for white-sounding names
n_w_calls = int(sum(data.call[data.race == 'w']))
sample_p_w = n_w_calls / len(data.call[data.race == 'w']) * 100 # working with percentages
print("The number of callbacks for white-sounding names is " + str(n_w_calls) + " out of "
      + str(len(data.call[data.race=='w'])) + " potential candidates (p1-hat = " + str(round(sample_p_w, 2)) +"%).")

# number of callbacks for white-sounding names
n_b_calls = int(sum(data.call[data.race == 'b']))
sample_p_b = n_b_calls / len(data.call[data.race == 'b']) * 100 # working with percentages
print("The number of callbacks for black-sounding names is " + str(n_b_calls) + " out of "
      + str(len(data.call[data.race=='b'])) + " potential candidates (p2-hat = " + str(round(sample_p_b, 2)) +"%).")

# difference between callback percentages based on race
sample_diff = abs(sample_p_w - sample_p_b)
print("The difference (p1-hat - p2-hat) between callback probabilties based on employer perceived race is {0}".format(u"\u0394") 
      + str(round(sample_diff, 2)) + "%.")

The total number of callbacks is 392 out of 4870 potential candidates (p-hat = 8.05%).
The number of callbacks for white-sounding names is 235 out of 2435 potential candidates (p1-hat = 9.65%).
The number of callbacks for black-sounding names is 157 out of 2435 potential candidates (p2-hat = 6.45%).
The difference (p1-hat - p2-hat) between callback probabilties based on employer perceived race is Δ3.2%.


***
#### 1.) What test is appropriate for this problem? Does CLT apply?

I think the Bernoulli Distribution is the right testing methodology here because we are dealing with a random variable (callback) which takes a value of 1 with probabilty ***p*** and a value of 0 with probability ***1 - p***.  

Yes, the Central Limit Theorem (CLT) does apply and this case meets the requirements as follows:
1. Randomness
2. Normal (*np [successes] >= 10* and *n(1-p) [failures] >= 10*)
3. Independent without replacment (*n <= 10% population* where our population is the entire US job market)

***
#### 2.) What are the null and alternate hypotheses?

**Null Hypothesis:** The probabilty for an employer callback does NOT depend on that employer's perceived race for the population *(p1 = p2)*.  
**Alternative Hypothesis:** The probabilty for an employer callback does have dependence on that emploter's perceived race for the population *(p1 != p2)*.  
**Confidence Interval**: 95%  
**Signifigence Level:** 5%

***
#### 3.) Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [4]:
# Bootstrapping Replication Function
def draw_bs_replicates(data, func, size=1):
    """Computes the bootstrap replicate(s) of a 1-dimensional numerical array"""
    bs_replicates = np.empty(size)
    
    for i in range(size):
        bs_sample = np.random.choice(data, size=len(data))
        bs_replicates[i] = func(bs_sample)
        
    return bs_replicates
    
# Empirical Cummulative Distribution Function
def ecdf(data):
    """Computes the Empirical Cummulative Distribution of a 1-dimensional numerical array"""
    n = len(data)
    x = np.sort(data)
    y = np.arange(1, n + 1) / n
    return x, y 

In [5]:
# Bootstrap Replication Statistical Analysis
# Plot 1
t_bs_replicates = draw_bs_replicates(data.call, np.mean, 1000) * 100
w_bs_replicates = draw_bs_replicates(data.call[data.race == 'w'], np.mean, 1000) * 100
b_bs_replicates = draw_bs_replicates(data.call[data.race == 'b'], np.mean, 1000) * 100

x_tbsr, y_tbsr = ecdf(t_bs_replicates)
x_wbsr, y_wbsr = ecdf(w_bs_replicates)
x_bbsr, y_bbsr = ecdf(b_bs_replicates)

x_tmen, y_tmen = np.array([[np.mean(t_bs_replicates), np.mean(t_bs_replicates)], [-0.05, 1.05]])
x_wmen, y_wmen = np.array([[np.mean(w_bs_replicates), np.mean(w_bs_replicates)], [-0.05, 1.05]])
x_bmen, y_bmen = np.array([[np.mean(b_bs_replicates), np.mean(b_bs_replicates)], [-0.05, 1.05]])

trace0 = go.Histogram(name="Total Bootstrap p", x=t_bs_replicates, histnorm="probability")
trace1 = go.Histogram(name="White Bootstrap p1", x=w_bs_replicates, histnorm="probability")
trace2 = go.Histogram(name="Black Bootstrap p2", x=b_bs_replicates, histnorm="probability")
trace3 = go.Scatter(name="Total Bootstrap p", x=x_tbsr, y=y_tbsr, mode="markers",
                    marker=dict(color="rgba(31, 119, 180, 1)"))
trace4 = go.Scatter(name="White Bootstrap p1", x=x_wbsr, y=y_wbsr, mode="markers",
                    marker=dict(color="rgba(255, 127, 14, 1)"))
trace5 = go.Scatter(name="Black Bootstrap p2", x=x_bbsr, y=y_bbsr, mode="markers",
                    marker=dict(color="rgba(44, 160, 44, 1)"))
trace6 = go.Scatter(name="Total Bootstrap p-mean", x=x_tmen, y=y_tmen, mode="lines",
                    line=dict(color="rgba(31, 119, 180, 1)"))
trace7 = go.Scatter(name="White Bootstrap p1-mean", x=x_wmen, y=y_wmen, mode="lines",
                    line=dict(color="rgba(255, 127, 14, 1)"))
trace8 = go.Scatter(name="Black Bootstrap p2-mean", x=x_bmen, y=y_bmen, mode="lines",
                    line=dict(color="rgba(44, 160, 44, 1)"))

fig = py.tools.make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.01)
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 1)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 1)
fig.append_trace(trace5, 2, 1)
fig.append_trace(trace6, 2, 1)
fig.append_trace(trace7, 2, 1)
fig.append_trace(trace8, 2, 1)

fig["layout"].update(
    plot_bgcolor="rgb(247,247,247)",
    legend=dict(font=dict(family="serif", size=12)),
    height=600,
    title="<b>Bootstrap Replication Callback Probability Sample Distributions</b>",
    titlefont=dict(family="serif", size=24),
    yaxis1=dict(title="<b>PDF</b>", titlefont=dict(family="serif", size=14),
               tickfont=dict(family="serif", size=14)), 
    yaxis2=dict(title="<b>ECDF</b>", titlefont=dict(family="serif", size=14), dtick=0.2,
               tickfont=dict(family="serif", size=14), range=[-0.05, 1.05]),
    xaxis1=dict(title="<b>Probability for Employer Callback, %</b>".format(u'\xb0'),
               titlefont=dict(family="serif", size=14), tickfont=dict(family="serif", size=14)))

py.offline.iplot(fig)

# Plot 2
diff_bs_replicates = w_bs_replicates - b_bs_replicates
x_dbsr, y_dbsr = ecdf(diff_bs_replicates)
x_dmen, y_dmen = np.array([[np.mean(diff_bs_replicates), np.mean(diff_bs_replicates)], [-0.05, 1.05]])
x_p0, y_p0 = np.array([[0, 0], [-0.05, 1.05]])

trace0 = go.Histogram(name="Bootstrap p1-p2", x=diff_bs_replicates, histnorm="probability")
trace1 = go.Scatter(name="Bootstrap p1-p2", x=x_dbsr, y=y_dbsr, mode="markers",
                    marker=dict(color="rgba(31, 119, 180, 1)"))
trace2 = go.Scatter(name="Bootstrap (p1-p2)mean", x=x_dmen, y=y_dmen, mode="lines",
                    marker=dict(color="rgba(31, 119, 180, 1)"))
trace3 = go.Scatter(name="Null Hypothesis p1-p2=0", x=x_p0, y=y_p0, mode="lines")

fig = py.tools.make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.01)
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 2, 1)
fig.append_trace(trace2, 2, 1)
fig.append_trace(trace3, 2, 1)

fig["layout"].update(
    plot_bgcolor="rgb(247,247,247)",
    legend=dict(font=dict(family="serif", size=12)),
    height=600,
    title="<b>Bootstrap Replication Difference Callback Probability Distribution</b>",
    titlefont=dict(family="serif", size=24),
    yaxis1=dict(title="<b>PDF</b>", titlefont=dict(family="serif", size=14),
               tickfont=dict(family="serif", size=14)), 
    yaxis2=dict(title="<b>ECDF</b>", titlefont=dict(family="serif", size=14), dtick=0.2,
               tickfont=dict(family="serif", size=14), range=[-0.05, 1.05]),
    xaxis1=dict(title="<b>Difference in Probability for Employer Callback, %</b>".format(u'\xb0'),
               titlefont=dict(family="serif", size=14), tickfont=dict(family="serif", size=14), range=[-0.5, 6.5]))

py.offline.iplot(fig)

# Bootstrap Stats
print("Bootstrap Replication Statistics for p1-p2:")

bs_conf_int = np.percentile(diff_bs_replicates, [2.5, 97.5])
print("     The margin of error is {0}".format(u"\u00B1") + 
            str(round((bs_conf_int[1] - bs_conf_int[0]) / 2, 2)) + "%.")
print("     The 95% confidence interval is " + str(round(bs_conf_int[0], 2)) + "% to " 
      + str(round(bs_conf_int[1], 2)) + "%.")

p0 = 0
bs_p = np.sum(diff_bs_replicates <= p0) / len(w_bs_replicates - b_bs_replicates) * 2 # two-tail p-value
print("     The p-value is " + str(round(bs_p * 100, 2)) + "%.")

# Frequentist Statistical Analysis
print()
print("Frequentist Statistics for p1-p2:")

# The margin of error for a 95% confidence interval
n = 2435 # both probability batches have the same number of trials n1 = n2 = n
z_star = stats.norm.ppf(0.975) # two-tail 95% interval has 2.5% on both sides
diff_stdv0 = (2 * sample_p_t * (100 - sample_p_t) / n) ** 0.5 # working with percentages
moe = z_star * diff_stdv0

print("     The margin of error is {0}".format(u"\u00B1") + str(round(moe, 2)) + "%.")

# The 95% confidence interval
conf_int = np.array([sample_diff - moe, sample_diff + moe])

print("     The 95% confidence interval is " + str(round(conf_int[0], 2)) + "% to " 
      + str(round(conf_int[1], 2)) + "%.")

# The p-value
z = (sample_diff - p0) / diff_stdv0
z_p = stats.norm.sf(abs(z)) * 2 # two-tail p-value

print("     The p-value is " + str(round(z_p * 100, 2)) + "%.")

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]



This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]



Bootstrap Replication Statistics for p1-p2:
     The margin of error is ±1.46%.
     The 95% confidence interval is 1.68% to 4.6%.
     The p-value is 0.0%.

Frequentist Statistics for p1-p2:
     The margin of error is ±1.53%.
     The 95% confidence interval is 1.68% to 4.73%.
     The p-value is 0.0%.


***
#### 4.) Write a story describing the statistical significance in the context of the original problem.

The resulting statistical inference test(s) above suggests that we *reject the null hypothesis* and that the probabilty for an employer callback might actually have dependence on that employer's perceived race of the job applicant.

***
#### 5.) Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

No, we cannot jump to the conclusion that race/name is the most important factor with our statictical analysis done thus far. There are other factors/categories (i.e. education, experience, gender, etc.) that need to be analyzed before we can make such a claim.